In [2]:
# Module Importations
from scipy.stats import reciprocal
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
import keras
import numpy as np
import time

print('keras.__version__=', keras.__version__)

Using TensorFlow backend.


In [3]:
# Project Module Importations
from src.data import load_data
from src.data import split_data
from src.models import keras_helpers

In [4]:
# Load dataset
original_dataset = load_data.load_motor_data()

In [5]:
# Split data into training / evaluation sets
training_set, evaluation_set = split_data.split_train_test(original_dataset, 0.8)

In [6]:
# Drop profile id data column
training_set = training_set.drop("profile_id", axis = 1)

In [7]:
# Create rotor target
rotor_training_data = training_set.drop("pm", axis = 1)
rotor_label_data = training_set["pm"].copy()

In [8]:
# Create train and test arrays
X_train, X_test, y_train, y_test = train_test_split(rotor_training_data, rotor_label_data, test_size = 0.2, random_state = 0)

In [10]:
# Multilayer Perceptron (Target - Rotor Temperature)

# Clear existing models
keras.backend.clear_session()

# Parameter distribution
param_distribs = { 
    "n_hidden": [1, 2, 4], 
    "n_neurons": np.arange(1, 100), 
    "learning_rate": reciprocal(3e-4, 3e-2)
    }

# Build model
wrapped_model = keras_helpers.wrap_model()

# Initialise Random Search
rnd_search_cv = RandomizedSearchCV(wrapped_model, param_distribs, n_iter = 10, cv = 3)

# Name model file
model_id = keras_helpers.name_model()
filepath_full = keras_helpers.make_save_string(model_id)

# Set save & earlystop callbacks
earlystop_cb = keras.callbacks.EarlyStopping(patience = 5)
checkpoint_cb = keras.callbacks.ModelCheckpoint(filepath = filepath_full, save_best_only = True)

# Train model
rnd_search_cv.fit( X_train, y_train, epochs = 50, validation_data =(X_test, y_test), callbacks =[checkpoint_cb, earlystop_cb])

Train on 106460 samples, validate on 39923 samples
Epoch 1/50
106460/106460 [==============================] - 4s 39us/step - loss: 0.2355 - val_loss: 0.2058
Epoch 2/50
106240/106460 [============================>.] - ETA: 0s - loss: 0.1954

KeyboardInterrupt: 

In [11]:
# Load model
filepath = r'C:\Developer\electric_motor_thermal_modelling\Models\YC_model_2020_07_23-12_37_34.h5'

model = keras.models.load_model(filepath)

model.evaluate(X_test, y_test)

39923/39923 [==============================] - 1s 18us/step


0.11522720813791532